# Top of the Atmosphere radiative imbalance
- FSNTOA: Net solar flux at top of atmosphere [W/m2]
- FSNT:   Net solar flux at top of model [W/m2]; positive down
- FLNT:   Net longwave flux at top of model [W/m2]; positive up

In [ ]:
import os
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - matplotlib.pyplot

In [ ]:
from paths import file_ex_atm_ctrl, path_results
from constants import spy
from timeseries import IterateOutputCESM
from xr_DataArrays import xr_AREA

In [ ]:
AREA = xr_AREA('atm')

In [ ]:
ds = xr.open_dataset(file_ex_atm_ctrl, decode_times=False)

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(16,5), sharey=True)
ds.FSNT.plot(ax=ax[0])
ds.FLNT.plot(ax=ax[1])
(ds.FSNT-ds.FLNT).plot(ax=ax[2])
plt.tight_layout()

In [ ]:
((ds.FSNT-ds.FLNT)*AREA).sum(dim=('lat', 'lon'))  # [W]

In [ ]:
%%time
# Flux Total Net Top of the model
# positive downward
def TOM_integral(run):
    """ top of the model annual global radiative imbalance """
    len_data = len(IterateOutputCESM(domain='atm', run=run, tavg='yrly', name='FSNT_FLNT'))
    FTNT = np.zeros((len_data))
    times = np.arange(len_data)
    for i, (y,m,s) in enumerate(IterateOutputCESM(domain='atm', run=run, tavg='yrly', name='FSNT_FLNT')):
        ds = xr.open_dataset(s, decode_times=False)
        FTNT[i] = ((ds.FSNT-ds.FLNT)*AREA).sum(dim=('lat', 'lon'))
        da = xr.DataArray(data=FTNT, coords={'time': times}, dims=('time'))
    return da

FTNT_ctrl = TOM_integral('ctrl')
FTNT_rcp  = TOM_integral('rcp')

In [ ]:
FTNT_ctrl.to_netcdf(f'{path_results}/TOA/TOM_ctrl.nc')
FTNT_rcp.to_netcdf( f'{path_results}/TOA/TOM_rcp.nc' )

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.axhline(0, c='k', lw=.5)
plt.plot(FTNT_ctrl.time+100, FTNT_ctrl*spy/1e21, lw=2, label='CTRL')
plt.plot(FTNT_rcp .time+200, FTNT_rcp *spy/1e21, lw=2, label='RCP')
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('TOM radiative imbalance [ZJ yr$^{-1}$]', fontsize=16)
plt.legend(loc=2, fontsize=16, frameon=False)
plt.tight_layout()
plt.savefig(f'{path_results}/TOA/TOA_global_timeseries')

In [ ]:
# linear detrending
ctrl_lf = np.polyfit(FTNT_ctrl.time, FTNT_ctrl, 1)
rcp_lf  = np.polyfit(FTNT_rcp.time , FTNT_rcp , 1)
ctrl_TOA_lin_detr = FTNT_ctrl-(ctrl_lf[1] + ctrl_lf[0]*FTNT_ctrl.time)
rcp_TOA_lin_detr  = FTNT_rcp -(rcp_lf[1]  + rcp_lf[0] *FTNT_rcp.time )

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.axhline(0, c='k', lw=.5)
plt.plot(ctrl_TOA_lin_detr.time+100, (ctrl_TOA_lin_detr)*spy/1e21, lw=.5, c=f'C{0}')
plt.plot(rcp_TOA_lin_detr .time+200, (rcp_TOA_lin_detr )*spy/1e21, lw=.5, c=f'C{1}')
plt.plot(ctrl_TOA_lin_detr.time+100, (ctrl_TOA_lin_detr.rolling({'time': 5}, center=True).mean())*spy/1e21, lw=2.5, c=f'C{0}', label='CTRL')
plt.plot(rcp_TOA_lin_detr .time+200, (rcp_TOA_lin_detr .rolling({'time': 5}, center=True).mean())*spy/1e21, lw=2.5, c=f'C{1}', label='RCP')
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('detr. TOM rad. imbalance [ZJ yr$^{-1}$]', fontsize=16)
plt.legend(loc=2, fontsize=16, frameon=False)
plt.tight_layout()
plt.savefig(f'{path_results}/TOA/TOA_global_timeseries_detr_smoothed')